In [ ]:
import os
from twilio.rest import Client
#from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [ ]:
TWILIO_ACCOUNT_SID ='XXXXXXXXXXXXXXXXXXXX'
TWILIO_AUTH_TOKEN ='XXXXXXXXXXXXXXXXXXXX'
PHONE_NUMBER ='+ XX XXX XXX XXX'
API_KEY_WAPI = 'XXXXXXXXXXXXXXXXXXXX'

# Armado de la URL

In [ ]:
query = 'place'
api_key = API_KEY_WAPI
url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=3&aqi=no&alerts=no'
url_clima

In [ ]:
response = requests.get(url_clima).json()

In [ ]:
response

In [ ]:
response.keys()

In [ ]:
response['forecast']['forecastday'][0].keys()

In [ ]:
len(response['forecast']['forecastday'][0]['hour'])

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

In [ ]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

# Dataframe

In [ ]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    dir_wind = (response['forecast']['forecastday'][0]['hour'][i]['wind_dir'])
    wind_mph = float(response['forecast']['forecastday'][0]['hour'][i]['wind_mph'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,dir_wind,wind_mph,rain,prob_rain

In [ ]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))

In [ ]:
datos[0]

In [ ]:
col = ['Fecha','Hora','Condicion','Temperatura','Direccion del Viento','Viento Mph','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

### EN MI CASO PONDRE UN RESUMEN DEL PRONOSTICO DEL DIA SIGUIENTE

In [ ]:
def get_time_range(hour):
    if 0 <= hour < 6:
        return 'De 0 a 6'
    elif 6 <= hour < 12:
        return 'De 6 a 12'
    elif 12 <= hour < 18:
        return 'De 12 a 18'
    else:
        return 'De 18 a 24'

In [ ]:
# Añadir una columna con el rango horario
df['Rango_Horario'] = df['Hora'].apply(get_time_range)

# Estadísticas requeridas
# Moda de la condición por rango horario
condicion_moda = df.groupby('Rango_Horario')['Condicion'].agg(pd.Series.mode)

# Promedio de viento por rango horario
viento_promedio = df.groupby('Rango_Horario')['Viento Mph'].mean()

# Moda de la dirección del viento por rango horario
direccion_moda = df.groupby('Rango_Horario')['Direccion del Viento'].agg(pd.Series.mode)

# Promedio de lluvia y probabilidad de lluvia por rango horario
lluvia_promedio = df.groupby('Rango_Horario')['Lluvia'].mean()
prob_lluvia_promedio = df.groupby('Rango_Horario')['prob_lluvia'].mean()

In [ ]:
# Mostrar resultados
#print("Moda de la condición por rango horario:")
#print(condicion_moda)

#print("\nPromedio de viento (mph) por rango horario:")
#print(viento_promedio)

#print("\nModa de la dirección del viento por rango horario:")
#print(direccion_moda)

#print("\nPromedio de lluvia por rango horario:")
#print(lluvia_promedio)

#print("\nPromedio de probabilidad de lluvia por rango horario:")
#print(prob_lluvia_promedio)

In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [ ]:
print(condicion_moda[0])

In [ ]:
'Hola!  El pronostico del tiempo en ' + query +' hoy '+ df['Fecha'][0] + ' de 0:00 a 6:00 el clima será ' + condicion_moda[0] +' con viento de : '+ str(viento_promedio[0]) + ' mph y dirección a ' +  direccion_moda[0] 

In [ ]:
#df_rain =  df[(df['Temperatura']>1) & (df['Hora']>6) & (df['Hora']< 22)]
#df_rain = df_rain[['Hora','Condicion']]
#df_rain.set_index('Hora', inplace = True)

In [ ]:
#df['Fecha'][0]

In [ ]:
#df_rain

In [ ]:
#'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

In [ ]:
PHONE_NUMBER

# Mensaje SMS desde Twilio

In [ ]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='Hola!  El pronostico del tiempo en ' + query +' hoy '+ df['Fecha'][2] + ' de 0:00 a 6:00 el clima será ' + condicion_moda[2] +' con viento de : '+ str(viento_promedio[2]) + ' mph y dirección a ' +  direccion_moda[2]  ,
                     from_=PHONE_NUMBER,
                     to='+XXXXXXX'
                 )

print('Mensaje Enviado ' + message.sid)